In [ ]:
import numpy as np
import pyomo.environ as pyo

def get_Q_b(problem_num):
    with open(f"resources/50d_instance{problem_num}.npy", 'rb') as f:
        Q = np.load(f)
        b = np.load(f)
        # global_sol = np.load(f)
    return Q, b

Q, w = get_Q_b(5)
n = len(Q)

# Create a model
model = pyo.ConcreteModel()

# Indices for the variables
model.I = range(n)

# Variables between 0 and 1
model.x = pyo.Var(model.I, within=pyo.UnitInterval)

# Objective function: 0.5 * x^T Q x + w^T x
def obj_rule(model):
    return 0.5 * sum(model.x[i] * Q[i, j] * model.x[j] for i in model.I for j in model.I) + sum(w[i] * model.x[i] for i in model.I)

model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Solver configuration
solver = pyo.SolverFactory('baron')

# Solve the model
results = solver.solve(model, tee=True)  # tee=True to display solver output

# Display the results
print("Status:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

for i in model.I:
    print(f"x[{i}] = {pyo.value(model.x[i])}")

In [ ]:
model = pyo.ConcreteModel()

# Index set for the variables
model.I = pyo.RangeSet(0, n-1)

# Variables, define range for x_i, for example, between 0 and pi
model.x = pyo.Var(model.I, bounds=(0, 1))

def objective_rule(model):
    # Summing over all i, j for the Q_ij * exp(x_i) * exp(x_j) terms
    expr1 = sum(0.5 * Q[i, j] * pyo.exp(model.x[i]) * pyo.exp(model.x[j]) for i in model.I for j in model.I)
    # Summing over all i for the w_i * exp(-x_i) terms
    expr2 = sum(w[i] * pyo.exp(-model.x[i]) for i in model.I)
    return expr1 + expr2

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)
solver = pyo.SolverFactory('baron')

# Solve the model
results = solver.solve(model, tee=True)

print("Status:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)
for i in model.I:
    print(f"x[{i}] = {pyo.value(model.x[i])}")